In [1]:
# import packages
import string
import copy
import re
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'%

In [40]:
import enum

class Tip(enum.Enum):
    ABSENT = 0
    PRESENT = 1
    CORRECT = 2

In [41]:
def score(secret, guess):
    # All characters that are not correct go into the usable pool.
    pool = collections.Counter(s for s, g in zip(secret, guess) if s != g)
    # Create a first tentative score by comparing char by char.
    score = []
    for secret_char, guess_char in zip(secret, guess):
        if secret_char == guess_char:
            score.append(Tip.CORRECT)
        elif guess_char in secret and pool[guess_char] > 0:
            score.append(Tip.PRESENT)
            pool[guess_char] -= 1
        else:
            score.append(Tip.ABSENT)

    return score

In [2]:
# read in existing western armenian dictionary
words_df = pd.read_csv('western_armenian_words.csv')
words_df.head()

,category,subcategory,english word,հայերէնի բար,pronounciation,lesson
0,food,drinks,tea,թէյ,tey,6.0
1,things,music,song,երգ,yerk,7.0
2,food,drinks,coffee,սուրճ,soorj,4.0
3,food,drinks,wine,գինի,kini,6.0
4,food,drinks,herbal tea,խոտաբոյսերու թէյ,khodapooyserov tey,7.0


In [3]:
aypoopen_upper = ['Ա','Բ','Գ','Դ','Ե','Զ','Է','Ը','Թ','Ժ','Ի','Լ','Խ','Ծ','Կ','Հ','Ձ','Ղ','Ճ','Մ','Յ','Ն','Շ','Ո','Չ','Պ','Ջ','Ռ','Ս','Վ','Տ','Ր','Ց','Ւ','Փ','Ք','Օ','Ֆ','և']
aypoopen_lower = ['ա','բ','գ','դ','ե','զ','է','ը','թ','ժ','ի','լ','խ','ծ','կ','հ','ձ','ղ','ճ','մ','յ','ն','շ','ո','չ','պ','ջ','ռ','ս','վ','տ','ր','ց','ւ','փ','ք','օ','ֆ','և']

aypoopen_dict = {aypoopen_lower[i]: aypoopen_upper[i] for i in range(len(aypoopen_lower))}

In [4]:
def caps(df):
    df['հայերէնի բար'] = df['հայերէնի բար'].replace(aypoopen_dict, regex=True)
    return df

In [5]:
# keep only the armenian words and the english translation
hay_df = words_df[['հայերէնի բար', 'english word']]
# keep only 5 character words
five_df = hay_df.loc[hay_df['հայերէնի բար'].str.len() == 5]
# remove words with non-letter characters
non_letter_df = five_df[~five_df['հայերէնի բար'].str.contains('։|՞|-')]
# make all characters capital
caps_df = caps(non_letter_df)
# get letters in each position
clean_df = caps_df
letter_pos = ['1l','2l','3l','4l','5l']
clean_df[letter_pos] = caps_df['հայերէնի բար'].str.split('',expand=True).iloc[:, 1:-1]
parig_df = clean_df.sort_values('հայերէնի բար')
parig_df.head()

,հայերէնի բար,english word,1l,2l,3l,4l,5l
410,ԱԿԱՆՋ,ear,Ա,Կ,Ա,Ն,Ջ
450,ԱԿՆՈՑ,glasses,Ա,Կ,Ն,Ո,Ց
76,ԱՂՋԻԿ,girl,Ա,Ղ,Ջ,Ի,Կ
658,ԱՂՏՈՏ,dirty,Ա,Ղ,Տ,Ո,Տ
356,ԱՄԵՆԱ,most,Ա,Մ,Ե,Ն,Ա


In [26]:
valid = False

while valid == False:
    count = 0
    guess = list(input("Please enter your guess: "))
    if len(guess) != 5:       
        print ('Please enter 5 characters')
    else:
        for x in guess:
            if x in aypoopen_upper or x in aypoopen_lower:
                count += 1
            else:
                print('That word is not in armenian. Try again')
                break
        if count == 5:
            valid = True
            print('Guess accepted')

Please enter your guess:  aveli


That word is not in armenian. Try again


Please enter your guess:  avel


Please enter 5 characters


Please enter your guess:  աւելի


Guess accepted


In [38]:
res = ''.join([aypoopen_dict.get(i, i) for i in list(guess)])
letter1, letter2, letter3, letter4, letter5 = list(res)

In [39]:
guess_df = parig_df[parig_df['հայերէնի բար'].str.contains()]

'Ա'

In [42]:
# https://www.inspiredpython.com/article/solving-wordle-puzzles-with-basic-python

def input_response():
    print("Type the color-coded reply from Wordle:")
    print("  G for Green")
    print("  Y for Yellow")
    print("  ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == WORD_LENGTH and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [ ]:
def match_word_vector(word, word_vector):
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        if letter not in v_letter:
            return False
    return True